In [37]:
import pandas as pd
import tensorflow as tf

# Automatically reload imported modules that are changed outside this notebook
# More pixels in figures
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200


In [38]:
tf.__version__

'2.5.0'

In [58]:
import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("new_test.tsv", sep="\t")
dev = pd.read_csv("new_dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
meta = pd.concat([train, test, dev])


In [59]:
meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en")))), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["locale"]  + "/clips/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = str(meta["Unnamed: 0"])
meta["target"] = meta["target"].map(targets)


workdir = "/tf/datasets/"


In [60]:
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"] 

23088    /tf/datasets/vox/ru_dev/nbC834J-lYM__U__S0---0...
23089    /tf/datasets/vox/ru_dev/cA2lgn3QohM__U__S13---...
23090    /tf/datasets/vox/ru_dev/aerIrz8ClqU__U__S28---...
23091    /tf/datasets/vox/ru_dev/hGiz9AhU1dM__U__S0---0...
23092    /tf/datasets/vox/ru_dev/HmAsKokkGyA__U__S10---...
                               ...                        
29020    /tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251--...
29021    /tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0...
29022    /tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0...
29023    /tf/datasets/vox/ru_dev/ya9uyy12vvM__U__S208--...
29024    /tf/datasets/vox/ru_dev/lUhjPr7Lcu8__U__S100--...
Name: path, Length: 5937, dtype: object

In [61]:
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] 

15966    /tf/datasets/vox/kz_dev/4QuKKCY5mAo__U__S151--...
15967    /tf/datasets/vox/kz_dev/YNTYeSoy0HM__U__S285--...
15968    /tf/datasets/vox/kz_dev/Gu-TfzMwS6g__U__S271--...
15969    /tf/datasets/vox/kz_dev/ASsJmutpNjg__U__S19---...
15970    /tf/datasets/vox/kz_dev/O0YlWuOb-YQ__U__S16---...
                               ...                        
23083    /tf/datasets/vox/kz_dev/23UJ403sOxA__U__S128--...
23084    /tf/datasets/vox/kz_dev/c2VaP6HXWS0__U__S41---...
23085    /tf/datasets/vox/kz_dev/xg8wrC8Gao4__U__S119--...
23086    /tf/datasets/vox/kz_dev/HIPlXLT3aYw__U__S149--...
23087    /tf/datasets/vox/kz_dev/BoTd0MzNHD0__U__S190--...
Name: path, Length: 7122, dtype: object

In [62]:
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] 

10106    /tf/datasets/vox/en_dev/Jjwf054bhTU__U__S0---0...
10107    /tf/datasets/vox/en_dev/6TEi0nJP5BE__U__S18---...
10108    /tf/datasets/vox/en_dev/Zq4wKUsd0ZM__U__S10---...
10109    /tf/datasets/vox/en_dev/8HzZXIchBhQ__U__S27---...
10110    /tf/datasets/vox/en_dev/-J43Gvlztc8__U__S37---...
                               ...                        
15961    /tf/datasets/vox/en_dev/SHU3RJ03QlM__U__S1---0...
15962    /tf/datasets/vox/en_dev/4wFO1Qqi3nQ__U__S0---0...
15963    /tf/datasets/vox/en_dev/4zWSmjij-hs__U__S54---...
15964    /tf/datasets/vox/en_dev/bi8wo64I57U__U__S0---0...
15965    /tf/datasets/vox/en_dev/pqAQ4JkeOZI__U__S2---0...
Name: path, Length: 5860, dtype: object

In [63]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"]

10000    /tf/datasets/vox/ru_test/jzh4S0PuCQo__U__S0---...
10001    /tf/datasets/vox/ru_test/PlqiRe7CEOY__U__S141-...
10002    /tf/datasets/vox/ru_test/6Siy_zm4gbc__U__S2---...
10003    /tf/datasets/vox/ru_test/w4DmYzuKSU0__U__S1---...
10004    /tf/datasets/vox/ru_test/yzy9Ps_6pMg__U__S0---...
                               ...                        
22102    /tf/datasets/vox/ru_test/oCp0C9NF_mA__U__S270-...
22103    /tf/datasets/vox/ru_test/6G25joynMLI__U__S0---...
22104    /tf/datasets/vox/ru_test/CKSJWnzoAPM__U__S2---...
22105    /tf/datasets/vox/ru_test/5dLGBKvH_gQ__U__S1---...
22106    /tf/datasets/vox/ru_test/pVLlwB6X-Ik__U__S0---...
Name: path, Length: 12107, dtype: object

In [64]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] 

22107    /tf/datasets/vox/kz_test/UjaSXBSJd3M__U__S53--...
22108    /tf/datasets/vox/kz_test/SZNVqf0AygU__U__S123-...
22109    /tf/datasets/vox/kz_test/1qwFgAt116E__U__S16--...
22110    /tf/datasets/vox/kz_test/KYCkqdclYaQ__U__S32--...
22111    /tf/datasets/vox/kz_test/g7NH6n9mcMo__U__S296-...
                               ...                        
36048    /tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...
36049    /tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...
36050    /tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...
36051    /tf/datasets/vox/kz_test/oCjW4Jy6azE__U__S110-...
36052    /tf/datasets/vox/kz_test/cwyX-Vjxep4__U__S30--...
Name: path, Length: 13946, dtype: object

In [65]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] 

0       /tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
1       /tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
2       /tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
3       /tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
4       /tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
                              ...                        
9995    /tf/datasets/vox/en_test/KLiy94kfZI4__U__S133-...
9996    /tf/datasets/vox/en_test/YTlliEr5LOA__U__S113-...
9997    /tf/datasets/vox/en_test/bSs0gNq6Kkc__U__S0---...
9998    /tf/datasets/vox/en_test/Da7c-BY6MDA__U__S2---...
9999    /tf/datasets/vox/en_test/VWvPndMo1F8__U__S24--...
Name: path, Length: 10000, dtype: object

In [66]:
meta.loc[meta["split"]=="test", "Unnamed: 0"] = meta.loc[meta["split"]=="test"]["path"]

In [67]:
meta["id"] = meta["Unnamed: 0"].apply(str)

In [68]:
meta.loc[meta["split"] == "test", "id"] = meta.loc[meta["split"] == "test"]["path"]

In [69]:
meta = meta.set_index("Unnamed: 0")
meta.loc[meta["split"]=="test"]

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/it/clips/common_voice_it_20015623.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,it,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/uk/clips/common_voice_uk_23554602.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,uk,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_20416266.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,tr,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...


In [70]:
meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 

In [71]:
workdir = "/tf/datasets/augmentedXvector/"
workdir

'/tf/datasets/augmentedXvector/'

In [72]:
meta.loc[meta["split"]=="test", "id"] = meta.loc[meta["split"]=="test"]["path"]
meta.loc[meta["split"]=="test"] 

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2.0,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2.0,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2.0,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25---0107.830-0127.780.mp3,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...,kz,test,0.0,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...
/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26---0236.830-0241.550.mp3,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...,kz,test,0.0,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...
/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78---0466.720-0470.860.mp3,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...,kz,test,0.0,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...


In [73]:
import scipy.signal

from lidbox.features import audio, cmvn
import lidbox.data.steps as ds_steps


TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def metadata_to_dataset_input(meta):
    return {
        "id": tf.constant(meta.id, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }

def read_mp3(x):
    s, r = audio.read_mp3(x["path"])
    out_rate = 16000
    s = audio.resample(s, r, out_rate)
    s = audio.peak_normalize(s, dBFS=-3.0)
    s = audio.remove_silence(s, out_rate)
    return dict(x, signal=s, sample_rate=out_rate)


def random_filter(x):
    def scipy_filter(s, N=10):
        b = np_rng.normal(0, 1, N)
        return scipy.signal.lfilter(b, 1.0, s).astype(np.float32), b
    s, _ = tf.numpy_function(
        scipy_filter,
        [x["signal"]],
        [tf.float32, tf.float64],
        name="np_random_filter")
    s = tf.cast(s, tf.float32)
    s = audio.peak_normalize(s, dBFS=-3.0)
    return dict(x, signal=s)


def random_speed_change(ds):
    return ds_steps.random_signal_speed_change(ds, min=0.9, max=1.1, flag=None)


def create_signal_chunks(ds):
    ds = ds_steps.repeat_too_short_signals(ds, 3200)
    ds = ds_steps.create_signal_chunks(ds, 3200, 800)
    return ds


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        S = audio.spectrograms(signals, rates[0])
        S = audio.linear_to_mel(S, rates[0])
        S = tf.math.log(S + 1e-6)
        S = cmvn(S, normalize_variance=False)
    return dict(x, logmelspec=S)

def pipeline_from_meta(data, split):
    if split == "train":
        data = data.sample(frac=1, random_state=np_rng.bit_generator)

    ds = (tf.data.Dataset
            .from_tensor_slices(metadata_to_dataset_input(data))
            .map(read_mp3, num_parallel_calls=TF_AUTOTUNE))

    if split == "train":
        return (ds
            .apply(random_speed_change)
           #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1)
            .map(random_filter, num_parallel_calls=TF_AUTOTUNE)
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch())
    else:
        return (ds
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch()
            #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1))


cachedir = os.path.join(workdir, "cache")

split2ds = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}

2021-06-13 07:01:05.895 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.900, 1.100]
2021-06-13 07:01:06.030 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-13 07:01:06.043 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.


2021-06-13 07:01:06.439 W tensorflow: Unresolved object in checkpoint: (root).layer-17


2021-06-13 07:01:06.440 W tensorflow: Unresolved object in checkpoint: (root).optimizer.iter


2021-06-13 07:01:06.441 W tensorflow: Unresolved object in checkpoint: (root).optimizer.beta_1


2021-06-13 07:01:06.441 W tensorflow: Unresolved object in checkpoint: (root).optimizer.beta_2


...

2021-06-13 07:01:06.450 W tensorflow: Unresolved object in checkpoint: (root).optimizer.beta_2


2021-06-13 07:01:06.450 W tensorflow: Unresolved object in checkpoint: (root).optimizer.decay


2021-06-13 07:01:06.451 W tensorflow: Unresolved object in checkpoint: (root).optimizer.learning_rate


2021-06-13 07:01:06.451 W tensorflow: A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


2021-06-13 07:01:06.697 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-13 07:01:06.710 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-13 07:01:07.093 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-13 07:01:07.105 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-13 07:01:07.440 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-13 07:01:07.453 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.


In [19]:
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Dense,
    Dropout,
    Input,
    Layer,
    SpatialDropout1D,
)
from tensorflow.keras.models import Model
import tensorflow as tf

def frame_layer(filters, kernel_size, strides, padding="causal", activation="relu", name="frame"):
    return Conv1D(filters, kernel_size, strides, padding=padding, activation=activation, name=name)


def segment_layer(units, activation="relu", name="segment"):
    return Dense(units, activation=activation, name=name)
class GlobalMeanStddevPooling1D(Layer):
    """
    Compute arithmetic mean and standard deviation of the inputs along the time steps dimension,
    then output the concatenation of the computed stats.
    """
    def call(self, inputs):
        means = tf.math.reduce_mean(inputs, axis=TIME_AXIS, keepdims=True)
        variances = tf.math.reduce_mean(tf.math.square(inputs - means), axis=TIME_AXIS)
        means = tf.squeeze(means, TIME_AXIS)
        stddevs = tf.math.sqrt(tf.clip_by_value(variances, STDDEV_SQRT_MIN_CLIP, variances.dtype.max))
        return tf.concat((means, stddevs), axis=TIME_AXIS)

def as_embedding_extractor(m):
    l = m.get_layer(name="segment1")
    l.activation = None
    return Model(inputs=m.inputs, outputs=l.output)

def frequency_attention(H, d_a=64, d_f=16):
    assert not H.shape[2] % d_f, "amount of frequency channels ({}) must be evenly divisible by the amount of frequency attention bins (d_f={})".format(H.shape[2], d_f)
    # Note, we assume that H.shape = (batch_size, T, d_h), but the paper assumes the timesteps come last
    x = Dense(d_a, activation="relu", use_bias=False, name="Wf_1")(H)
    F_A = Dense(d_f, activation="softmax", use_bias=False, name="Wf_2")(x)
    # Apply frequency attention on d_f bins
    F_A = Reshape((F_A.shape[1] or -1, F_A.shape[2], 1), name="expand_bin_weight_dim")(F_A)
    H_bins = Reshape((H.shape[1] or -1, d_f, H.shape[2] // d_f), name="partition_freq_bins")(H)
    H_bins = Multiply(name="freq_attention")([F_A, H_bins])
    # Merge weighted frequency bins
    H_weighted = Reshape((H.shape[1] or -1, H.shape[2]), name="merge_weighted_bins")(H_bins)
    return H_weighted

In [24]:
from tensorflow.keras.layers import (
    Activation,
    Dense,
    Input,
)
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Conv2D,
    Dropout,
    Dense,
    GaussianNoise,
    Input,
    Layer,
    LSTM,
    Multiply,
    Reshape,
)
from tensorflow.keras.models import Model
import tensorflow as tf

# Assuming spectral features (Batch, Time, Channels), where freq. channels are always last
TIME_AXIS = 1
STDDEV_SQRT_MIN_CLIP = 1e-10

def create(input_shape, num_outputs, output_activation="log_softmax", freq_attention_bins=60):
    inputs = Input(shape=input_shape, name="input")
    x = SpatialDropout1D(0.7, name="channel_dropout")(inputs)
    x = frame_layer(512, 5, 1, name="frame1")(x)
    x = frame_layer(512, 3, 2, name="frame2")(x)
    x = frame_layer(512, 3, 3, name="frame3")(x)
    x = frame_layer(512, 1, 1, name="frame4")(x)
    x = frame_layer(1500, 1, 1, name="frame5")(x)

    x = frequency_attention(x, d_f=freq_attention_bins)

    x = GlobalMeanStddevPooling1D(name="stats_pooling")(x)

    x = segment_layer(512, name="segment1")(x)
    x = segment_layer(512, name="segment2")(x)

    outputs = Dense(num_outputs, name="output", activation=None)(x)
    if output_activation:
        outputs = Activation(getattr(tf.nn, output_activation), name=str(output_activation))(outputs)
    return Model(inputs=inputs, outputs=outputs, name="x-vector-frequency-attention")

In [28]:


def create_model(num_freq_bins=20, num_labels=len(np.unique(meta.target))):
    m = create(
        input_shape=[None, num_freq_bins],
        num_outputs=4)
    m.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)
    return m

with tf.device("GPU"):
    model = create_model()
    model.summary()
   

callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(cachedir, "tensorboard", model.name),
        update_freq="epoch",
        write_images=True,
        profile_batch=0,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(cachedir, "model", model.name),
        monitor='val_loss',
        save_weights_only=True,
        save_best_only=True,
        verbose=1,
    ),
]


def as_model_input(x):
    return x["logmelspec"], x["target"]



Model: "x-vector-frequency-attention"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
channel_dropout (SpatialDropout (None, None, 20)     0           input[0][0]                      
__________________________________________________________________________________________________
frame1 (Conv1D)                 (None, None, 512)    51712       channel_dropout[0][0]            
__________________________________________________________________________________________________
frame2 (Conv1D)                 (None, None, 512)    786944      frame1[0][0]                     
_______________________________________________________________________

In [29]:
cachedir = "/tf/datasets/augmentedXvector/cache"


In [30]:
import pandas as pd

from lidbox.util import predict_with_model, classification_report
from lidbox.visualize import draw_confusion_matrix


def load_trained_model():
    model = create_model()
    print(os.path.join(cachedir, "model", model.name))
    model.load_weights(os.path.join(cachedir, "model", model.name))
    return model


def display_classification_report(report):
    for m in ("avg_detection_cost", "avg_equal_error_rate", "accuracy"):
        print("{}: {:.3f}".format(m, report[m]))

    lang_metrics = pd.DataFrame.from_dict(
        {k: v for k, v in report.items() if k in lang2target})
    lang_metrics["mean"] = lang_metrics.mean(axis=1)
    display(lang_metrics.T)

    fig, ax = draw_confusion_matrix(report["confusion_matrix"], lang2target)

model = load_trained_model()

def predict_with_ap_loss(x):
    with tf.device("GPU"):
        # Generate language vector for input spectra
        language_vector = model(x["input"], training=False)
        print(language_vector)
        # Predict languages by computing distances to reference directions
        return x["id"], model.loss.predict(language_vector)




/tf/datasets/augmentedXvector/cache/model/x-vector-frequency-attention


In [31]:

def predictions_to_dataframe(ids, predictions):
    return (pd.DataFrame.from_dict({"id": ids, "prediction": predictions})
            #.set_index("id", drop=True, verify_integrity=True)
            #.sort_index()
           )

def predict_with_model(model, ds, predict_fn=None):
    """
    Map callable model over all batches in ds, predicting values for each element at key 'input'.
    """
    if predict_fn is None:
        def predict_fn(x):
            with tf.device("GPU"):
                return x["id"], model(x["input"], training=False)

    ids = []
    predictions = []
    for id, pred in ds.map(predict_fn, num_parallel_calls=TF_AUTOTUNE).unbatch().as_numpy_iterator():
        ids.append(id.decode("utf-8"))
        predictions.append(pred)

    return predictions_to_dataframe(ids, predictions)

## Test on vox

In [32]:
meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 

In [74]:
meta = meta.set_index("id")

In [75]:
meta.loc[meta["split"] == "test"]

,path,locale,split,target
id,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2.0
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2.0
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2.0
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2.0
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2.0
...,...,...,...,...
/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25---0107.830-0127.780.mp3,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...,kz,test,0.0
/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26---0236.830-0241.550.mp3,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...,kz,test,0.0
/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78---0466.720-0470.860.mp3,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...,kz,test,0.0


Ru sst

In [76]:
chunk2pred = predict_with_model(
    model=model,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [77]:
chunk2pred

,id,prediction
0,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,"[-0.07002593, -5.7154145, -2.9013617, -4.6683955]"
1,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,"[-1.1671677, -3.3739388, -0.5356533, -2.6705878]"
2,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,"[-2.5124385, -1.915247, -0.3730118, -2.4893112]"
3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,"[-0.9713418, -1.0899353, -1.9481032, -1.9474114]"
4,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,"[-1.0492101, -1.7267493, -2.0425928, -1.0722663]"
...,...,...
334514,/tf/datasets/vox/kz_test/cwyX-Vjxep4__U__S30--...,"[-0.32531023, -3.5286946, -2.813746, -1.6693275]"
334515,/tf/datasets/vox/kz_test/cwyX-Vjxep4__U__S30--...,"[-0.17609802, -4.480584, -2.3373587, -2.9270456]"
334516,/tf/datasets/vox/kz_test/cwyX-Vjxep4__U__S30--...,"[-0.45195174, -3.5194185, -1.8153256, -1.764844]"
334517,/tf/datasets/vox/kz_test/cwyX-Vjxep4__U__S30--...,"[-0.9917979, -3.170356, -1.4174447, -1.064879]"


In [78]:
chunk2pred = chunk2pred.set_index("id")

In [79]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0003.940-0020.570.mp3,"[-1.0297503, -3.851068, -1.5921042, -1.3939339]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0743.730-0757.100.mp3,"[-1.734242, -3.4777792, -1.0620568, -1.0570086]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0650.920-0661.560.mp3,"[-1.4066441, -2.2093089, -1.649218, -1.0521832]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0692.790-0704.510.mp3,"[-1.2719195, -2.2588575, -1.7421018, -1.2674958]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0705.010-0711.610.mp3,"[-1.813632, -2.107427, -1.5414699, -0.9765296]"
...,...
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0219.180-0230.690.mp3,"[-2.3263788, -1.9312198, -2.472705, -1.0607697]"
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0230.690-0247.370.mp3,"[-2.7114298, -1.2576344, -3.0530705, -1.3916095]"
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0247.370-0257.750.mp3,"[-2.3065178, -1.8669779, -2.068025, -1.0563043]"


In [80]:
from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.45      0.46      0.45     13946
          ru       0.78      0.25      0.37     12107
          en       0.98      0.24      0.39     10000
       other       0.00      0.00      0.00         0

    accuracy                           0.33     36053
   macro avg       0.55      0.24      0.30     36053
weighted avg       0.71      0.33      0.41     36053



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
